In [1]:
import math
import random
import pandas as pd
import numpy as np
import os

from typing import List, TypedDict
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from openai import OpenAI

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)

## Загрузка данных

In [3]:
data = pd.read_json("data_final_for_dls_new.jsonl", lines=True)
data.columns = ['text', 'address', 'name', 'norm_name_ru', 'permalink', 'prices_summarized', 'relevance', 'reviews_summarized', 'relevance_new']
train_data = data[570:]
eval_data = data[:570]
train_data = train_data[train_data["relevance"] != 0.1].reset_index(drop=True)
eval_data = eval_data[eval_data["relevance"] != 0.1].reset_index(drop=True)

Проводилась небольшая подготовка данных. В колонку `summarized` объединялись данные из 'norm_name_ru', 'prices_summarized', 'reviews_summarized'. "Лишние" колонки затем удалялись, для более удобного восприятия и отображения датасета.

In [4]:
eval_data['reviews_summarized'] = eval_data['reviews_summarized'].fillna(";")
eval_data['prices_summarized'] = eval_data['prices_summarized'].fillna(";")
eval_data['reviews_summarized'] = eval_data['reviews_summarized'].str.split(r'[\n|]').str[0].str.strip()
eval_data['summarized'] = (eval_data['norm_name_ru'] + ' ; ' + eval_data['prices_summarized'] + ' ; ' + eval_data['reviews_summarized'])
eval_data.drop(columns=['norm_name_ru', 'prices_summarized', 'reviews_summarized', 'permalink'], inplace=True)

In [5]:
# eval_data.to_excel('eval_data_summarized.xlsx')

In [6]:
eval_data.head(2)

,text,address,name,relevance,relevance_new,summarized
0,сигары,"Москва, Дубравная улица, 34/29",Tabaccos; Магазин Tabaccos; Табаккос,1.0,1.0,Магазин табака и курительных принадлежностей ;...
1,кальянная спб мероприятия,"Санкт-Петербург, Большой проспект Петроградско...",PioNero; Pionero; Пицца Паста бар; Pio Nero; P...,0.0,0.0,Кафе ; PioNero предлагает разнообразные блюда ...


In [7]:
eval_data.shape

(500, 6)

### deepseek-chat-v3 - финальный промпт для классификации

In [179]:
# Вызываем модель и передаем ей инструменты
load_dotenv()
client = OpenAI(
    base_url="https://api.ai-mediator.ru/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [180]:
# Определение типов для структуры данных
class OrganizationData(TypedDict):
    user_query: str
    organization_address: str
    organization_name: str
    summary: str
    classification: str

Тут представлена версия промпта, доработанная в ходе построения агента. Это финальная версия для классификационного этапа.

In [181]:
# Системный промпт для классификации

SYSTEM_PROMPT = """Ты помощник для классификации организаций по релевантности пользовательскому запросу.
Оцени, подходит ли организация к данному запросу. Отвечай только "relevant" или "irrelevant".

**Правила:**
1. Сначала определи, есть ли в запросе географические указания (город, район, адрес). Если нет - пункт 2 не применяй.
2. Учитывай географическую принадлежность к городу (ТОЛЬКО если в запросе явно указан город)
3. Учитывай совпадение по ключевым словам ("налоговая", "больница", "ресторан" и т.д.):
   - анализируй тип деятельности организации и соответствие запросу — даже если совпадений по словам немного, 
   но смысловой контекст пересекается, отмечай как relevant.
4. Учитывай совпадение по номеру (если цифры есть в запросе)
5. Оцени другие возможные критерии, исходя из которых можно сделать вывод "relevant" или "irrelevant".

**Примеры:**
Запрос "налоговая 5007" → проверяй только ключевые слова и номер
Запрос "больница в Королёве" → проверяй ключевые слова и геолокацию
"""

**в `classify_organization` прописать модель**

In [182]:
def classify_organization(row: pd.Series) -> str:
    """Определяет релевантность организации запросу пользователя"""
    
    # Формируем пользовательский промпт
    user_prompt = f"""
    ЗАПРОС ПОЛЬЗОВАТЕЛЯ: {row['text']}
    
    ИНФОРМАЦИЯ ОБ ОРГАНИЗАЦИИ:
    - Адрес: {row['address']}
    - Название: {row['name']}
    - Сводка: {row['summarized']}
    """
    
    # Вызов модели
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    
    # Получаем и возвращаем классификацию
    return response.choices[0].message.content.strip().lower()

In [183]:
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Обрабатывает DataFrame с данными организаций"""
    
    # Создаем копию DataFrame для результатов
    result_df = df.copy()
    
    # Добавляем столбец с классификацией
    tqdm.pandas(desc="Classifying organizations")
    result_df['llm_rel'] = df.progress_apply(classify_organization, axis=1)
    
    return result_df

In [184]:
# Обрабатываем данные
classified_data = process_dataframe(eval_data)

Classifying organizations:   0%|          | 0/500 [00:00<?, ?it/s]

In [185]:
classified_data['llm_rel'] = np.where(classified_data['llm_rel'].str.contains('irrelevant'), 0, 1)

In [186]:
# classified_data

Результат сохраняем в файл

In [ ]:
# classified_data.to_excel('V3_test_final_prompt.xlsx')

In [ ]:
classified_data = pd.read_excel('V3_test_final_prompt.xlsx')

In [189]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(classified_data['relevance'], classified_data['llm_rel'])
accuracy_new = accuracy_score(classified_data['relevance_new'], classified_data['llm_rel'])
print(f"Accuracy: {accuracy:.2f}", f"Accuracy_new: {accuracy_new:.2f}", sep='\n')

Accuracy: 0.66
Accuracy_new: 0.77


**Вывод**
- deepseek-chat-v3-0324: 24 руб за 500 запросов. 8м 30сек. Accuracy = 0.66/0.77 

На измененном промпте данная модель показала accuracy  равный 0.77 на новой разметке. Это на 8% выше, чем эта-же модель на стартовом промпте. Вместе с тем, изменения в финальном промте вовсе не носят радикальный характер, а лишь слегка изменяют/дополняют стартовый.  
Однако данная модель LLM очень "чутко"/"капризно" реагирует на любые изменения в промптах. Это важный аспект, который явно следует учитывать при выборе модели LLM под те или иные задачи. 